In [ ]:
!pip install requests sodapy pandas ydata-profiling

  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sodapy import Socrata
from google.colab import userdata

In [ ]:
APP_TOKEN = "DFuj3IhWdzABB5sSwWj6Fw1T3"
client = Socrata("data.sfgov.org", APP_TOKEN)

# Spending & Revenue Dataset

[Data Source](https://data.sfgov.org/City-Management-and-Ethics/Spending-and-Revenue/bpnb-jwfb/about_data)


## Data Ingestion

In [ ]:
srd = client.get("xdgd-c79v", limit=280000)
df = pd.DataFrame.from_records(srd)
df.describe()

,fiscal_year,revenue_or_spending,related_govt_unit,organization_group_code,organization_group,department_code,department,program_code,program,character_code,...,object,sub_object_code,sub_object,fund_type_code,fund_type,fund_code,fund,fund_category_code,fund_category,budget
count,272922,272922,272922,272922,272922,272922,272922,272922,272922,272922,...,272922,272922,272922,272922,272922,272922,272922,272900,272900,272922
unique,14,2,4,7,7,67,118,383,361,97,...,671,3527,2852,36,30,496,491,8,11,89042
top,2018,Spending,NO,02,"Public Works, Transportation & Commerce",DPH,DPW GSA - Public Works,OPR,Operating,MAND_FRING_BEN,...,Health Service,01701,Unemployment Insurance,1G,General Fund,1GAGF,General Fund,1,Operating,0
freq,39755,225879,144365,92369,92369,27417,21017,90759,90759,37652,...,20349,3880,7475,59477,110784,58579,58579,142755,142755,101228


## Data Profiling

,fiscal_year,revenue_or_spending,related_govt_unit,organization_group_code,organization_group,department_code,department,program_code,program,character_code,...,object,sub_object_code,sub_object,fund_type_code,fund_type,fund_code,fund,fund_category_code,fund_category,budget
count,272922,272922,272922,272922,272922,272922,272922,272922,272922,272922,...,272922,272922,272922,272922,272922,272922,272922,272900,272900,272922
unique,14,2,4,7,7,67,118,383,361,97,...,671,3527,2852,36,30,496,491,8,11,89042
top,2018,Spending,NO,02,"Public Works, Transportation & Commerce",DPH,DPW GSA - Public Works,OPR,Operating,MAND_FRING_BEN,...,Health Service,01701,Unemployment Insurance,1G,General Fund,1GAGF,General Fund,1,Operating,0
freq,39755,225879,144365,92369,92369,27417,21017,90759,90759,37652,...,20349,3880,7475,59477,110784,58579,58579,142755,142755,101228


In [ ]:

df.loc[df["program"] == "County Education Services"]

In [ ]:
budget_df = df.filter(["fiscal_year", "revenue_or_spending", "organization_group", "department", "budget"])
budget_df = budget_df.astype({'budget':'double'})
budget_df.head(10)

In [ ]:
spending_2020_df = budget_2020_df.loc[budget_2020_df["revenue_or_spending"]=="Spending"]
revenue_2020_df = budget_2020_df.loc[budget_2020_df["revenue_or_spending"]=="Revenue"]

In [ ]:
dept_spend_2020 = spending_2020_df.groupby(["department"]).sum()
dept_revenue_2020 = revenue_2020_df.groupby(["department"]).sum()
org_spend_2020 = spending_2020_df.groupby(["organization_group"]).sum()
org_revenue_2020 = revenue_2020_df.groupby(["organization_group"]).sum()

In [ ]:
org_spend_2020.plot.pie(y="budget", figsize=(10,10))

# Employee Compensation Dataset
[Data Source](https://data.sfgov.org/City-Management-and-Ethics/Employee-Compensation/88g8-5mnd)

In [ ]:
ecd = client.get("88g8-5mnd", limit=9000000)

In [ ]:
df = pd.DataFrame.from_records(ecd)
df.describe()

,organization_group_code,job_family_code,job_code,year_type,year,organization_group,department_code,department,union_code,union,...,employee_identifier,salaries,overtime,other_salaries,total_salary,retirement,health_and_dental,other_benefits,total_benefits,total_compensation
count,881800,881800,881800,881800,881800,881800,881798,881798,881620,881621,...,881800,881800,881800,881800,881800,881800,881800,881800,881800,881800
unique,7,59,1230,2,11,7,62,111,70,129,...,490655,649938,351400,393942,797439,587462,255411,619117,789043,848403
top,02,2300,9163,Fiscal,2019,"Public Works, Transportation & Commerce",DPH,Public Health,790,"SEIU, Local 1021, Misc",...,0,0,0,0,250,0,0,0.00,0.00,268.74
freq,277896,93537,56158,461597,89053,277896,197449,124356,246502,206298,...,49,9836,263437,174851,614,69477,124664,973,454,120


# Vendor Payments Dataset
[Data Source](https://www.google.com/url?q=https://data.sfgov.org/City-Management-and-Ethics/Vendor-Payments-Purchase-Order-Summary-/p5r5-fd7g&sa=D&source=editors&ust=1701989218435124&usg=AOvVaw3C99V-mS_btastX_gu4LlO)


In [ ]:
vpd = client.get("p5r5-fd7g", limit=1100000)

In [ ]:
df_vpd = pd.DataFrame.from_records(vpd)
df_vpd.describe()

,fiscal_year,related_govt_units,organization_group_code,organization_group,department_code,department,program_code,program,character_code,character,...,fund_code,fund,fund_category_code,fund_category,purchase_order,vendor,vouchers_paid,vouchers_pending,vouchers_pending_retainage,encumbrance_balance
count,1100000,1100000,1100000,1100000,1100000,1100000,1099905,1099905,1100000,1100000,...,1100000,1100000,1096172,1096172,1100000,1100000,1100000,1100000,1100000,165343
unique,12,4,7,7,64,70,427,404,26,23,...,291,286,7,10,692622,64690,438785,95309,103497,65452
top,2018,No,02,"Public Works, Transportation & Commerce",DPH,DPH Public Health,OPR,Operating,540,Materials & Supplies,...,1GAGF,General Fund,1,Operating,Direct Payments,CARDINALHEALTH MEDICAL PRODUCTS & SVCS,0.00,0.00,0.00,0
freq,165343,927984,522060,522060,200152,200152,155947,155947,489597,573686,...,287953,287953,819819,819819,162266,58997,122086,810439,785426,62181


# Survey Data
[Data Source](https://data.sfgov.org/widgets/nufj-bfbw?mobile_redirect=true)


In [ ]:
sd = client.get("nufj-bfbw", limit=50000)

In [ ]:
df_sd = pd.DataFrame.from_records(sd)
df_sd.describe()

,year,weight,general_rate,inf_water,inf_sewer,inf_cleanpwr,inf_cleanpwr_qual,inf_clean,inf_sidecond,inf_stcond,...,u311_rate_web,dem_mentdis_yes,inf_watsew,inf_clean_side,inf_clean_st,rec_grounds,u311_info_phone,u311_svc_phone,u311_info_web,u311_svc_web
count,42720,42685,40555,5771,5565,1540,905,5848,10489,31585,...,321,4296,4599,20177,25828,32314,2960,2375,1680,1446
unique,18,3728,5,5,5,2,5,5,5,5,...,5,2,5,5,5,5,5,5,5,5
top,2011,1,3,5,4,1,4,4,4,3,...,4,0,4,4,4,4,4,4,4,4
freq,3979,5867,18046,2783,2213,938,428,1604,3957,10589,...,142,4071,2068,7088,9765,15848,1196,922,633,534
